<a href="https://colab.research.google.com/github/cemararengganis/first-contributions/blob/master/DS3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install spark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [52.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 k

In [ ]:
#set environment
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [ ]:
#inisialisasi
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
#ambil Spark COntext
from pyspark import SparkContext
sc = spark.sparkContext;

In [ ]:
#membuat RDD dengan struktur seragam
data = sc.parallelize([('Amir', 22), ('Badu', 23), ('Cica',4), ('Doni', 12), ('Amir',
9)]).collect()
data

[('Amir', 22), ('Badu', 23), ('Cica', 4), ('Doni', 12), ('Amir', 9)]

In [ ]:
#membuat RDD dengan struktur campuran
data_campur = sc.parallelize([('Pesawat', 'Garuda',20), {'Ertiga': 210000000},
['Jakarta','Bandung', 5]]).collect()
data_campur

[('Pesawat', 'Garuda', 20), {'Ertiga': 210000000}, ['Jakarta', 'Bandung', 5]]

In [ ]:
#ambil spark context
from pyspark import SparkContext
sc = spark.sparkContext;

In [ ]:
#load data
df = spark.read.json('kontak.json')

In [ ]:
#tampil data
df.show(5)

+-------+----+
|   nama|usia|
+-------+----+
|   Amir|null|
|   Badu|  30|
|   Doni|  30|
|Singgih|  33|
|   Dedy|  43|
+-------+----+
only showing top 5 rows



In [ ]:
#baca csv
df2 = spark.read.csv('karyawan.csv',inferSchema=True,header=True)
df2.show(5)

+-------+-------+
|   nama|   gaji|
+-------+-------+
|   Amir|7000000|
|   Badu|6000000|
|   Doni|6000000|
|Singgih|6600000|
|   Dedy|4300000|
+-------+-------+
only showing top 5 rows



In [ ]:
#menggunakan agregat
print(df.agg({"usia": "max"}).collect())
from pyspark.sql import functions as F
print(df.agg(F.max('usia')).collect())

[Row(max(usia)=43)]
[Row(max(usia)=43)]


In [ ]:
#membuat alias
from pyspark.sql.functions import *
kontak = df.alias("kontak")
karyawan = df2.alias("karyawan")
gabung = kontak.join(karyawan, col("kontak.nama") == col("karyawan.nama"), 'inner')
final = gabung.select("kontak.nama", "karyawan.gaji", "kontak.usia")
final.show()

+-------+--------+----+
|   nama|    gaji|usia|
+-------+--------+----+
|   Amir| 7000000|null|
|   Badu| 6000000|  30|
|   Doni| 6000000|  30|
|Singgih| 6600000|  33|
|   Dedy| 4300000|  43|
|  Nuzul| 7300000|null|
|Mustofa|15000000|  27|
|   Umar| 7200000|  36|
|Arindra| 7800000|  39|
|  Apang| 7400000|  37|
+-------+--------+----+



In [ ]:
#beberapa fungsi sama dengan RDD
#cache
final.cache()
#collect
print(final.collect())
#count
print(final.count())

[Row(nama='Amir', gaji=7000000, usia=None), Row(nama='Badu', gaji=6000000, usia=30), Row(nama='Doni', gaji=6000000, usia=30), Row(nama='Singgih', gaji=6600000, usia=33), Row(nama='Dedy', gaji=4300000, usia=43), Row(nama='Nuzul', gaji=7300000, usia=None), Row(nama='Mustofa', gaji=15000000, usia=27), Row(nama='Umar', gaji=7200000, usia=36), Row(nama='Arindra', gaji=7800000, usia=39), Row(nama='Apang', gaji=7400000, usia=37)]
10


In [ ]:
#tampil nama kolom
final.columns

['nama', 'gaji', 'usia']

In [ ]:
#korelasi
final.corr('usia','gaji')

-0.1198946577057265

In [ ]:
#komputasi statistik kolom numerik
final.describe().show()

+-------+----+----------------+-----------------+
|summary|nama|            gaji|             usia|
+-------+----+----------------+-----------------+
|  count|  10|              10|                8|
|   mean|null|       7460000.0|           34.375|
| stddev|null|2833607.82984049|5.343554194407634|
|    min|Amir|         4300000|               27|
|    max|Umar|        15000000|               43|
+-------+----+----------------+-----------------+



In [ ]:
#distinct
print(final.select('gaji').count())
print(final.select('gaji').distinct().count())
print(final.select('gaji').distinct().show())

10
9
+--------+
|    gaji|
+--------+
| 7000000|
| 7800000|
|15000000|
| 7200000|
| 7400000|
| 4300000|
| 6600000|
| 7300000|
| 6000000|
+--------+

None


In [ ]:
#drop kolom
print(df.columns)
print(df2.columns)
print(df.drop('usia').show())

['nama', 'usia']
['nama', 'gaji']
+-------+
|   nama|
+-------+
|   Amir|
|   Badu|
|   Doni|
|Singgih|
|   Dedy|
|  Nuzul|
|Mustofa|
|   Umar|
|Arindra|
|  Apang|
+-------+

None


In [ ]:
df.show(5)

+-------+----+
|   nama|usia|
+-------+----+
|   Amir|null|
|   Badu|  30|
|   Doni|  30|
|Singgih|  33|
|   Dedy|  43|
+-------+----+
only showing top 5 rows



In [ ]:
#https://finance.yahoo.com/quote/TLKM.JK/history?period1=1096329600&period2=159122880
#0&interval=1d&filter=history&frequency=1d
#load ke memori
df = spark.read.csv('TLKM.JK.csv',inferSchema=True,header=True)

In [ ]:
#tampil 5 data awal
df.show(5)

+-------------------+----------+----------+----------+----------+----------+---------+
|               Date|      Open|      High|       Low|     Close| Adj Close|   Volume|
+-------------------+----------+----------+----------+----------+----------+---------+
|2004-09-28 00:00:00|835.000000|835.000000|825.000000|825.000000|123.351845| 90095000|
|2004-09-29 00:00:00|825.000000|825.000000|810.000000|825.000000|123.351845| 29370000|
|2004-09-30 00:00:00|825.000000|830.000000|825.000000|830.000000|124.099464| 42927500|
|2004-10-01 00:00:00|830.000000|845.000000|825.000000|840.000000|125.594643|108387500|
|2004-10-04 00:00:00|845.000000|880.000000|845.000000|870.000000|130.080124|424210000|
+-------------------+----------+----------+----------+----------+----------+---------+
only showing top 5 rows



In [ ]:
#info tabel
df.printSchema()
df.count()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- Volume: string (nullable = true)



3897

In [ ]:
#ubah tipe data
df = df.withColumn("Open", df.Open.cast('double'))
df = df.withColumn("High", df.High.cast('double'))
df = df.withColumn("Low", df.Low.cast('double'))
df = df.withColumn("Close", df.Close.cast('double'))
df = df.withColumn("Adj Close", df['Adj Close'].cast('double'))
df = df.withColumn("Volume", df.Volume.cast('integer'))

In [ ]:
#cek skema berubah
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)



In [ ]:
#cek apakah ada yang null
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when( col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+----+----+---+-----+---------+------+
|Date|Open|High|Low|Close|Adj Close|Volume|
+----+----+----+---+-----+---------+------+
|   0|  36|  36| 36|   36|       36|    36|
+----+----+----+---+-----+---------+------+



In [ ]:
#drop null
dfclean = df.na.drop(how='any')

In [ ]:
#cek ulang null
dfclean.select([count(when( col(c).isNull(), c)).alias(c) for c in
dfclean.columns]).show()

+----+----+----+---+-----+---------+------+
|Date|Open|High|Low|Close|Adj Close|Volume|
+----+----+----+---+-----+---------+------+
|   0|   0|   0|  0|    0|        0|     0|
+----+----+----+---+-----+---------+------+



In [ ]:
#tampilkan statistik
dfclean.describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+-------------------+
|summary|              Open|              High|               Low|             Close|         Adj Close|             Volume|
+-------+------------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              3861|              3861|              3861|              3861|              3861|               3861|
|   mean|2424.5754985754984|2452.6120176120176|2397.2286972286975|  2425.00777000777|1804.0317654027438|9.757185266122766E7|
| stddev|1083.9830930036037|1092.4516781623754|1076.2375509513736|1084.2149926092493|1475.6031910021748| 6.07356066496236E7|
|    min|             820.0|             825.0|             810.0|             825.0|        123.351845|                  0|
|    max|            4800.0|            4840.0|            4780.0|            4800.0|       4693.016113|          808005000|


In [ ]:
import requests

headers = {
    'authority': 'www.olx.co.id',
    'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
    'accept': '*/*',
    'x-newrelic-id': 'VQMGU1ZVDxABU1lbBgMDUlI=',
    'sec-ch-ua-mobile': '?0',
    'x-panamera-fingerprint': '0a3a9f247abc7e8bc173124c78093aa5#1617784056249',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.olx.co.id/items/q-mobil',
    'accept-language': 'en-US,en;q=0.9',
    'cookie': 'bm_sz=9FC41A746160CBE196C13B6A97AEE8FF~YAAQTRdLF6J6AnN4AQAAd5pxqwuk/9nqbvRq3DRYEiaN0avDcWkLYs/Bu0sLK8GTR4LLgRVXIvX+lqDdUZ5PXWrZCTfEwBdDAuyGDTyrim6svxGiCajSI4kPWgVYbv9paLG1uhXLKbj6JfNkEsW5fRRGQgA3fKtK6R9Z5FzECMGDnYZVgKEVxTFgDU6ky5A=; ak_bmsc=1248118460D1ABB546423237C8D04132174B174DF6060000F66C6D6098856575~plYDQvS9Ld5aE0TXmMbpbWZX/eofXblzGVkNL9rPcAnfKeeMUpVi+3gxo0/DIEAqZyxl27Z5sR2ikf2r0K0HInlkEXixDnFndF+I/tH8hUxGRzmcI/k1ZzuEe7MMkWcCKezp0FKT4lE36QhJ0sjFQl84gTzUJuMlpdddmwba7gGBDK4xaPdrRV+THKlF9+KgjmH4AdSbZmvOB3SHeYJdzUjOEbmGDpexF8yHzyNz/PzIgQhLtX0SWEs2piQdOwMmF6; _abck=94F40CAAA3033DA12706A06E37C36883~0~YAAQTRdLF8x6AnN4AQAAFqxxqwUAxrWCI6zfSZVlxwd7iwN0LZqbKYzBNOpxmsIl420qG0i3eH9Q+rLnHFk+lUTfARgvicOqrdMrvkUYefQGe7NWUbZuCNFpWgxa0jwKWh3maGXksxOb/iLY7kEm0xrDBO1X483McdkIpPXlV0OuyiroEZY9i+2MyQYPqakdaSoG7pEB4xYz6uqdjpfWsc5Pv3u8lNPDBTUrHyXsyGEwIB5DCniATq7BFTM58dSMVZk09EGgJvLwmh0aQcJimSAGCGSwcYnu0Gm0DfRrdtlLPFaNsPrm6bjZl3a9KXFE5WuNp9YqpUACkTw5VRUMZwEjMKYdpCG8Q6WxdPkMOu3tgxAJSSllEL79t5QcPPgvX9/MyOD/aUjZ9Hyb5ebs+ZAAjHtOYGk=~-1~||-1||~-1; ldTd=true; lqstatus=1617785257||||; laquesis=pan-46706@b#pan-49638@a; laquesisff=; laquesissu=; _ga=GA1.3.189924337.1617784057; _gid=GA1.3.1431383857.1617784057; _rtb_user_id=e1810f77-ca16-5d41-4f93-d19fd0e9c8f4; _gcl_au=1.1.862897736.1617784057; WZRK_G=0c5bbfe21e6a4f30be81c920fca01fba; __gads=ID=117409c4e118a75d:T=1617784058:S=ALNI_MbM7fFeiARoEBOXrNv66pJQkMnUAA; g_state={"i_p":1617791261716,"i_l":1}; onap=178ab71ac70x8aba0ff-1-178ab71ac70x8aba0ff-15-1617785871; bm_sv=8B20648BB3DAD6FEA35753416F732D62~OiWg+P6fsRDiolHQNJm1f+M6V3ArddZcy0qMrgYJApeiv2uKUDpxRrTproYE+kg7nnXvZjnx2jrTmbBZCmHC8HUU9Cw4wb5o1ybnOzj+iUW3hhDXQIuCyVuygZ7V1a7n5PgMhUyMbLMbpuSXPRdNwhmWmI/Ep19ThgYAyz5YHr4=; WZRK_S_W6K-746-995Z=%7B%22p%22%3A1%2C%22s%22%3A1617784057%2C%22t%22%3A1617784177%7D',
}

params = (
    ('facet_limit', '100'),
    ('location', '1000001'),
    ('location_facet_limit', '20'),
    ('page', '1'),
    ('query', 'mobil'),
    ('spellcheck', 'true'),
    ('user', '178ab71ac70x8aba0ff'),
)

response = requests.get('https://www.olx.co.id/api/relevance/search', headers=headers, params=params)

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.get('https://www.olx.co.id/api/relevance/search?facet_limit=100&location=1000001&location_facet_limit=20&page=1&query=mobil&spellcheck=true&user=178ab71ac70x8aba0ff', headers=headers)


In [ ]:
# konversi ke json
data_mobil = response.json()

In [ ]:
# cek panjang dengan len
len(data_mobil['data'])

20

In [ ]:
# simpan data
data_mobil = data_mobil['data']

In [ ]:
data_mobil[0]

{'car_body_type': 'hatchback',
 'category_id': '198',
 'certified_car': False,
 'created_at': '2021-03-23T15:15:57+07:00',
 'created_at_first': '2021-02-05T21:45:32+07:00',
 'description': 'OA-Mobil19-229\n\nHonda Jazz 1.5 RS A/T 2016 Hitam\n\nHarga Rp 179.000.000,- (Credit)\nHarga Cash (Call)\n\nSimulasi Kredit :\nTenor 4 Tahun DP 20 Juta. Angsuran 48 x 5.367.000*\nTenor 5 Tahun DP 20 Juta. Angsuran 60 x 4.715.000*\n*Harga dapat berubah sewaktu-waktu\n\nFREE BENEFIT (keuntungan) tambahan jika anda membeli mobil di OLX Autos - Jual - Beli - Tukar Tambah\n\nSpesifikasi Kendaraan:\n- STNK dan BPKB Asli dan Akur (Atas nama Pribadi)\n- Pajak Oktober 2019\n- Plat Nomor Ganjil\n- Warna Hitam\n- Odometer 68.507 km\n- Interior Original Fabric Model\n- Head Unit Original dan Terawat\n- Velg dan Ban Original\n- Kaki-kaki Terawat\n- Bebas Banjir\n- Bebas Tabrak\n- Mobil rawatan Bengkel Resmi Honda\n\nUntuk meyakinkan anda, kami berikan free Test drive\n\nJl. Raden Saleh Blok: 10 No. 25, RT.5/RW.1

In [ ]:
# target yang kita ambil
# title
# deskripsi
# harga
# favorite
import pandas as pd


In [ ]:
item = []
for i in range(len(data_mobil)):
  wadah = {} #dictionary
  wadah['judul'] = data_mobil[i]['title']
  wadah['deskripsi']= data_mobil[i]['description']
  wadah['harga']= data_mobil[i]['price']['value']['raw']
  wadah['favorit']= data_mobil[i]['favorites']['count']
  item.append(wadah)

item_df = pd.DataFrame(item)

 

In [ ]:
item_df.sort_values('harga',ascending=False).head()

,judul,deskripsi,harga,favorit
1,Porsche Macan 2.0 2014 Akhir White On Black Fu...,"Plat Ganjil, Full Option, NIK 2014 Akhir, Paja...",815000000.0,12
16,TOYOTA FORTUNER VRZ 4X2 2018 ISTIMEWA #DAKAR #...,TOYOTA FORTUNER VRZ 4X2 2018 ISTIMEWA\n\nNIK :...,430000000.0,20
8,Toyota Alphard G ATPM 2012,Toyota Alpgard G ATPM 2012 Hitam\nTangan 1\nOr...,325000000.0,0
0,[OLXAutos] Honda Jazz 1.5 RS A/T 2016 Hitam,OA-Mobil19-229\n\nHonda Jazz 1.5 RS A/T 2016 H...,179000000.0,7
3,Great New Xenia type R Sporty,Bismillah\nDijual Great New Xenia type R Sport...,167000000.0,13


In [ ]:
# Selenium
!apt update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [ ]:
!pip install selenium

In [ ]:
# config
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver' )
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
browser = webdriver.Chrome('chromedriver', chrome_options = chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
# akses
browser.get(url)

NameError: ignored

In [3]:
# tentukan situs yg diakses
url = "https://www.olx.co.id/"

In [ ]:
# akses
browser.get(url)

In [ ]:
# simpan halaman
def simpan_html(html):
  f = open('cek.html', "a")
  f.seek(0)
  f.truncate()
  f.write(html)
  f.close()

In [ ]:
simpan_html(browser.page_source)

In [ ]:
# akses searchbar
browser.find_element_by_id('v-omnisearch__input').send_keys("topi")
browser.find_element_by_class_name('v-omnisearch__submit').click()


In [ ]:
simpan_html(browser.page_source)